# Long Short Term Memory(LSTM)-  Time-series prediction model

- <b>Using Long Short Term Memory (LSTM) to buid a Time-series prediction model to predict the "Diuresis" on 27th March.</b> 
- <b>With the value of Diuresis on 27th March and substuting in the test dataset predicting the Infect_Prob using Support Vector Regression(SVR).</b>
- <b>Output is Infect_prob with People_Id is in the form of excel file "part2.xlsx"</b>

In [1]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences)-1:
            break
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [3]:
dt = pd.read_excel("Train_dataset.xlsx", sheet_name="Diuresis_TS")

In [4]:
dt=dt.drop('people_ID',axis=1)

In [5]:
dt2=dt.iloc[:,:].values

In [6]:
dt2

array([[ 441.      ,  544.2     ,  669.64    , ..., 1002.2416  ,
        1215.68992 , 1472.627904],
       [ 151.      ,  197.2     ,  253.24    , ...,  399.9256  ,
         492.91072 ,  606.892864],
       [ 416.      ,  515.2     ,  632.44    , ...,  947.7736  ,
        1150.32832 , 1395.793984],
       ...,
       [ 177.      ,  228.4     ,  288.28    , ...,  448.4632  ,
         551.15584 ,  676.787008],
       [ 352.      ,  438.4     ,  542.68    , ...,  814.6792  ,
         990.61504 , 1204.138048],
       [ 181.      ,  230.2     ,  290.44    , ...,  455.2936  ,
         559.35232 ,  686.622784]])

In [7]:
dt2=dt2.transpose()

In [8]:
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")

In [9]:
dt2

array([[ 441.      ,  151.      ,  416.      , ...,  177.      ,
         352.      ,  181.      ],
       [ 544.2     ,  197.2     ,  515.2     , ...,  228.4     ,
         438.4     ,  230.2     ],
       [ 669.64    ,  253.24    ,  632.44    , ...,  288.28    ,
         542.68    ,  290.44    ],
       ...,
       [1002.2416  ,  399.9256  ,  947.7736  , ...,  448.4632  ,
         814.6792  ,  455.2936  ],
       [1215.68992 ,  492.91072 , 1150.32832 , ...,  551.15584 ,
         990.61504 ,  559.35232 ],
       [1472.627904,  606.892864, 1395.793984, ...,  676.787008,
        1204.138048,  686.622784]])

In [10]:
n_steps = 3

In [12]:
X, Y = split_sequences(dt2, n_steps)

In [13]:
n_features = X.shape[2]

print(X.shape[2])
print(X)

10714
[[[ 441.       151.       416.      ...  177.       352.       181.     ]
  [ 544.2      197.2      515.2     ...  228.4      438.4      230.2    ]
  [ 669.64     253.24     632.44    ...  288.28     542.68     290.44   ]]

 [[ 544.2      197.2      515.2     ...  228.4      438.4      230.2    ]
  [ 669.64     253.24     632.44    ...  288.28     542.68     290.44   ]
  [ 821.368    321.688    776.728   ...  362.136    665.816    366.328  ]]

 [[ 669.64     253.24     632.44    ...  288.28     542.68     290.44   ]
  [ 821.368    321.688    776.728   ...  362.136    665.816    366.328  ]
  [1002.2416   399.9256   947.7736  ...  448.4632   814.6792   455.2936 ]]

 [[ 821.368    321.688    776.728   ...  362.136    665.816    366.328  ]
  [1002.2416   399.9256   947.7736  ...  448.4632   814.6792   455.2936 ]
  [1215.68992  492.91072 1150.32832 ...  551.15584  990.61504  559.35232]]]


In [14]:
Y

array([[ 821.368   ,  321.688   ,  776.728   , ...,  362.136   ,
         665.816   ,  366.328   ],
       [1002.2416  ,  399.9256  ,  947.7736  , ...,  448.4632  ,
         814.6792  ,  455.2936  ],
       [1215.68992 ,  492.91072 , 1150.32832 , ...,  551.15584 ,
         990.61504 ,  559.35232 ],
       [1472.627904,  606.892864, 1395.793984, ...,  676.787008,
        1204.138048,  686.622784]])

## Defining Model & Fitting the Model

In [15]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')
model.fit(X, Y, epochs=400, verbose=0)

In [16]:
series=dt2[4:7,:]

In [17]:
series

array([[1002.2416  ,  399.9256  ,  947.7736  , ...,  448.4632  ,
         814.6792  ,  455.2936  ],
       [1215.68992 ,  492.91072 , 1150.32832 , ...,  551.15584 ,
         990.61504 ,  559.35232 ],
       [1472.627904,  606.892864, 1395.793984, ...,  676.787008,
        1204.138048,  686.622784]])

In [18]:
series = series.reshape((1, n_steps, n_features))
temp = model.predict(series, verbose=0)

In [19]:
temp

array([[1823.0388 ,  739.5979 , 1723.0999 , ...,  825.66486, 1483.2754 ,
         835.0345 ]], dtype=float32)

# Training and predicting Infect_Prob of Test Dataset Using SVR

In [20]:
ds = pd.read_excel("Train_dataset.xlsx",sheet_name = "Train_dataset")
ds1 = pd.read_excel("Test_dataset.xlsx")
ds2= pd.read_excel("Test_dataset.xlsx")

In [21]:
ds.head()

,people_ID,Region,Gender,Designation,Name,Married,Children,Occupation,Mode_transport,cases/1M,...,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,Insurance,salary,FT/month,Infect_Prob
0,1,Bhubaneshwar,Female,Mrs,mansi,YES,1.0,Farmer,Public,2,...,93.0,233.0,82.0,58.0,27,7,3600000.0,1300000,2.0,49.135010
1,2,Bhubaneshwar,Female,Mrs,riya masi,YES,2.0,Farmer,Walk,2,...,56.0,328.0,89.0,68.0,5,6,1600000.0,400000,1.0,51.147880
2,3,Bhubaneshwar,Female,Mrs,sunita,NO,1.0,Cleaner,Public,2,...,137.0,213.0,77.0,43.0,40,6,3400000.0,900000,1.0,73.224000
3,4,Bhubaneshwar,Female,Mrs,anjali @ babli,YES,1.0,Driver,Car,2,...,167.0,275.0,64.0,60.0,27,7,700000.0,2300000,1.0,48.779225
4,5,Bhubaneshwar,Female,Mrs,champa karketta,NO,2.0,Manufacturing,Car,2,...,153.0,331.0,71.0,64.0,32,7,3200000.0,1100000,1.0,87.868800


In [22]:
ds1.head()

,people_ID,Region,Gender,Designation,Name,Married,Children,Occupation,Mode_transport,cases/1M,...,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,Insurance,salary,FT/month
0,5942,Delhi,Female,Mrs,smt rekha prajapat,YES,2,Driver,Public,4,...,153,196,240,85,53,17,3,3900000,1300000,1
1,18664,Delhi,Male,Mr,nirmal,YES,2,Legal,Walk,4,...,95,138,241,81,61,2,5,1800000,1300000,1
2,5603,Delhi,Female,Mrs,pinky,YES,2,Sales,Car,4,...,40,166,236,88,47,24,3,5000000,2000000,2
3,5649,Delhi,Female,Mrs,pooja @aafrin,YES,2,Sales,Car,4,...,78,83,211,87,52,13,6,3100000,600000,2
4,5099,Delhi,Female,Mrs,anjali,YES,2,Business,Car,4,...,109,207,312,94,68,39,5,2300000,1500000,1


# Preprocessing of Train Dataset

### Dropping of attributes

In [23]:
ds.drop(["salary","Insurance","Married","Children","people_ID","Designation","Name"], axis = 1, inplace = True)

In [24]:
ds.head()

,Region,Gender,Occupation,Mode_transport,cases/1M,Deaths/1M,comorbidity,Age,Coma score,Pulmonary score,...,Diuresis,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,FT/month,Infect_Prob
0,Bhubaneshwar,Female,Farmer,Public,2,0,Hypertension,68,8,<400,...,441.0,154.0,93.0,233.0,82.0,58.0,27,7,2.0,49.135010
1,Bhubaneshwar,Female,Farmer,Walk,2,0,Diabetes,64,15,<100,...,NaN,121.0,56.0,328.0,89.0,68.0,5,6,1.0,51.147880
2,Bhubaneshwar,Female,Cleaner,Public,2,0,None,19,13,<300,...,416.0,124.0,137.0,213.0,77.0,43.0,40,6,1.0,73.224000
3,Bhubaneshwar,Female,Driver,Car,2,0,Coronary Heart Disease,33,9,<200,...,410.0,98.0,167.0,275.0,64.0,60.0,27,7,1.0,48.779225
4,Bhubaneshwar,Female,Manufacturing,Car,2,0,Diabetes,23,7,<400,...,390.0,21.0,153.0,331.0,71.0,64.0,32,7,1.0,87.868800


### Filling Missing Values

In [57]:
ds_null = ds.columns[ds.isnull().any()]
ds[ds_null].isnull().sum()

Series([], dtype: float64)

In [26]:
ds_categ = list(ds.select_dtypes(exclude = ["number"]).columns)
ds_categ

['Region',
 'Gender',
 'Occupation',
 'Mode_transport',
 'comorbidity',
 'Pulmonary score',
 'cardiological pressure']

In [27]:
ds["Mode_transport"].fillna(method ='ffill', inplace = True)
ds["comorbidity"].fillna(method ='ffill', inplace = True)
ds["cardiological pressure"].fillna(method ='ffill', inplace = True)
ds["Diuresis"].fillna(method ='ffill', inplace = True)
ds["Platelets"].fillna(method ='ffill', inplace = True)
ds["HBB"].fillna(method ='ffill', inplace = True)
ds["d-dimer"].fillna(method ='ffill', inplace = True)
ds["Heart rate"].fillna(method ='ffill', inplace = True)
ds["HDL cholesterol"].fillna(method ='ffill', inplace = True)
ds["FT/month"].fillna(method ='ffill', inplace = True)
ds["Occupation"].fillna(method ='ffill', inplace = True)

In [28]:
ds.head()

,Region,Gender,Occupation,Mode_transport,cases/1M,Deaths/1M,comorbidity,Age,Coma score,Pulmonary score,...,Diuresis,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,FT/month,Infect_Prob
0,Bhubaneshwar,Female,Farmer,Public,2,0,Hypertension,68,8,<400,...,441.0,154.0,93.0,233.0,82.0,58.0,27,7,2.0,49.135010
1,Bhubaneshwar,Female,Farmer,Walk,2,0,Diabetes,64,15,<100,...,441.0,121.0,56.0,328.0,89.0,68.0,5,6,1.0,51.147880
2,Bhubaneshwar,Female,Cleaner,Public,2,0,None,19,13,<300,...,416.0,124.0,137.0,213.0,77.0,43.0,40,6,1.0,73.224000
3,Bhubaneshwar,Female,Driver,Car,2,0,Coronary Heart Disease,33,9,<200,...,410.0,98.0,167.0,275.0,64.0,60.0,27,7,1.0,48.779225
4,Bhubaneshwar,Female,Manufacturing,Car,2,0,Diabetes,23,7,<400,...,390.0,21.0,153.0,331.0,71.0,64.0,32,7,1.0,87.868800


In [29]:
ds_categ = list(ds.select_dtypes(exclude = ["number"]).columns)
ds_categ

['Region',
 'Gender',
 'Occupation',
 'Mode_transport',
 'comorbidity',
 'Pulmonary score',
 'cardiological pressure']

### Converting Categorical Value to Continous Value using Label Encoder

In [30]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in ds_categ:
    print(ds[i].unique())
    ds[i] = le.fit_transform(ds[i])

['Bhubaneshwar' 'Chandigarh' 'Chennai' 'Delhi' 'Bengaluru']
['Female' 'Male']
['Farmer' 'Cleaner' 'Driver' 'Manufacturing' 'Clerk' 'Sales' 'Business'
 'Researcher' 'Legal']
['Public' 'Walk' 'Car']
['Hypertension' 'Diabetes' 'None' 'Coronary Heart Disease']
['<400' '<100' '<300' '<200']
['Normal' 'Stage-02' 'Elevated' 'Stage-01']


In [31]:
ds.head()

,Region,Gender,Occupation,Mode_transport,cases/1M,Deaths/1M,comorbidity,Age,Coma score,Pulmonary score,...,Diuresis,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,FT/month,Infect_Prob
0,1,0,4,1,2,0,2,68,8,3,...,441.0,154.0,93.0,233.0,82.0,58.0,27,7,2.0,49.135010
1,1,0,4,2,2,0,1,64,15,0,...,441.0,121.0,56.0,328.0,89.0,68.0,5,6,1.0,51.147880
2,1,0,1,1,2,0,3,19,13,2,...,416.0,124.0,137.0,213.0,77.0,43.0,40,6,1.0,73.224000
3,1,0,3,0,2,0,0,33,9,1,...,410.0,98.0,167.0,275.0,64.0,60.0,27,7,1.0,48.779225
4,1,0,6,0,2,0,1,23,7,3,...,390.0,21.0,153.0,331.0,71.0,64.0,32,7,1.0,87.868800


# Preprocessing Test Data

In [32]:
ds1.drop(["salary","Insurance","Married","Children","people_ID","Designation","Name"], axis = 1, inplace = True)

In [33]:
ds1.head()

,Region,Gender,Occupation,Mode_transport,cases/1M,Deaths/1M,comorbidity,Age,Coma score,Pulmonary score,cardiological pressure,Diuresis,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,FT/month
0,Delhi,Female,Driver,Public,4,1,Diabetes,52,3,<200,Stage-01,388,153,196,240,85,53,17,3,1
1,Delhi,Male,Legal,Walk,4,1,Diabetes,53,11,<200,Stage-01,409,95,138,241,81,61,2,5,1
2,Delhi,Female,Sales,Car,4,1,Diabetes,35,9,<100,Stage-02,440,40,166,236,88,47,24,3,2
3,Delhi,Female,Sales,Car,4,1,None,31,12,<300,Elevated,206,78,83,211,87,52,13,6,2
4,Delhi,Female,Business,Car,4,1,Diabetes,51,6,<200,Stage-01,229,109,207,312,94,68,39,5,1


In [34]:
ds1_null = ds1.columns[ds1.isnull().any()]
ds1[ds1_null].isnull().sum()

Series([], dtype: float64)

In [35]:
ds1_categ = list(ds1.select_dtypes(exclude = ["number"]).columns)
ds1_categ

['Region',
 'Gender',
 'Occupation',
 'Mode_transport',
 'comorbidity',
 'Pulmonary score',
 'cardiological pressure']

### Converting Categorical Value to Continous Value using Label Encoder

In [36]:
le = LabelEncoder()
for i in ds1_categ:
    print(ds1[i].unique())
    ds1[i] = le.fit_transform(ds1[i])

['Delhi' 'Hyderabad' 'Kolkata' 'Mumbai' 'Pune' 'Thiruvananthapuram']
['Female' 'Male']
['Driver' 'Legal' 'Sales' 'Business' 'Manufacturing' 'Researcher' 'Clerk'
 'Cleaner' 'Farmer']
['Public' 'Walk' 'Car']
['Diabetes' 'None' 'Hypertension' 'Coronary Heart Disease']
['<200' '<100' '<300' '<400']
['Stage-01' 'Stage-02' 'Elevated' 'Normal']


In [37]:
ds1.head()

,Region,Gender,Occupation,Mode_transport,cases/1M,Deaths/1M,comorbidity,Age,Coma score,Pulmonary score,cardiological pressure,Diuresis,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,FT/month
0,0,0,3,1,4,1,1,52,3,1,2,388,153,196,240,85,53,17,3,1
1,0,1,5,2,4,1,1,53,11,1,2,409,95,138,241,81,61,2,5,1
2,0,0,8,0,4,1,1,35,9,0,3,440,40,166,236,88,47,24,3,2
3,0,0,8,0,4,1,3,31,12,2,0,206,78,83,211,87,52,13,6,2
4,0,0,0,0,4,1,1,51,6,1,2,229,109,207,312,94,68,39,5,1


#### <b>Substituting the new Diuresis value in Test Dataset form LSTM model.<b>

In [38]:
for i in range(len(temp)):
    print(i)
    ds1['Diuresis'][i]=temp[i][0]

0


C:\Users\kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
ds1.head()

,Region,Gender,Occupation,Mode_transport,cases/1M,Deaths/1M,comorbidity,Age,Coma score,Pulmonary score,cardiological pressure,Diuresis,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,FT/month
0,0,0,3,1,4,1,1,52,3,1,2,1823,153,196,240,85,53,17,3,1
1,0,1,5,2,4,1,1,53,11,1,2,409,95,138,241,81,61,2,5,1
2,0,0,8,0,4,1,1,35,9,0,3,440,40,166,236,88,47,24,3,2
3,0,0,8,0,4,1,3,31,12,2,0,206,78,83,211,87,52,13,6,2
4,0,0,0,0,4,1,1,51,6,1,2,229,109,207,312,94,68,39,5,1


In [40]:
ds.shape

(10714, 21)

In [41]:
ds1.shape

(14498, 20)

### Using MinMax Scaler for Normalizing

In [42]:
X = ds.iloc[:,0:20]
y = ds.iloc[:,[20]]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler2=MinMaxScaler()
X_train = scaler.fit_transform(X)
y = np.reshape(np.array(y), (-1,1))
y_train = scaler2.fit_transform(y)

In [43]:
X_train.shape

(10714, 20)

In [44]:
y_train.shape

(10714, 1)

In [45]:
A = ds1.iloc[:,0:20]

In [46]:
scaler = MinMaxScaler()
X_test = scaler.fit_transform(A)

In [47]:
X_test.shape

(14498, 20)

# Support Vector Regression

<b>I have just used rbf kernel with gamma and C as a parameter. I didn't go for with other kernels and various values of gamma and C because the other kernels were taking lot of time to train and fit the dataset. I got the mean absolute better in linear kernel but it is taking a lot of time in training the dataset set compare to rbf kernel. Tried Grid Search also to get the best kernel and parameters but due to system capacity it was taking a lot of time to give the best model. Therefore, I have used the rbf kernel to build the model and train the dataset</b>

In [48]:
from sklearn.svm import SVR

In [49]:
svr1 = SVR(kernel = 'rbf',gamma=1, C = 10000)

In [50]:
svr1.fit(X_train,y_train)

C:\Users\kumar\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=10000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [51]:
predictions = svr1.predict(X_test)

In [52]:
predictions

array([0.32432572, 0.25439328, 0.38916057, ..., 0.31602042, 0.36161314,
       0.42935267])

In [53]:
temp1=predictions.reshape(-1,1)
temp1=scaler2.inverse_transform(temp1)

In [54]:
temp1

array([[51.45479833],
       [46.67544764],
       [55.88576708],
       ...,
       [50.88719421],
       [54.00310978],
       [58.63259201]])

In [55]:
testpplid=ds2.iloc[:,0].values

In [56]:
import xlsxwriter 
  
workbook = xlsxwriter.Workbook('part2.xlsx') 
worksheet = workbook.add_worksheet() 
  
row = 0
row_id=0
column_id = 0
column=1
  
for i in range(len(temp1)) : 
  
    
    worksheet.write(row, column_id, testpplid[i])
    worksheet.write(row, column, temp1[i]) 
    
    row += 1
      
workbook.close() 